# DQN (Deep Q-Net) using Q-learning using batchnorm- TEST

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import gym
import numpy as np

In [2]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0
Default GPU Device: /device:GPU:0


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

In [3]:
# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [4]:
env.reset()
rewards, states, actions, dones = [], [], [], []
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    states.append(state)
    rewards.append(reward)
    actions.append(action)
    dones.append(done)
    print('state, action, reward, done, info')
    print(state, action, reward, done, info)
    if done:
        print('state, action, reward, done, info')
        print(state, action, reward, done, info)
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        dones.append(done)

state, action, reward, done, info
[-0.02761163 -0.18210196 -0.03206792  0.32645303] 0 1.0 False {}
state, action, reward, done, info
[-0.03125367  0.01346154 -0.02553886  0.02383225] 1 1.0 False {}
state, action, reward, done, info
[-0.03098444  0.20894025 -0.02506221 -0.27679781] 1 1.0 False {}
state, action, reward, done, info
[-0.02680564  0.01418466 -0.03059817  0.00787633] 0 1.0 False {}
state, action, reward, done, info
[-0.02652194  0.20973176 -0.03044064 -0.29430149] 1 1.0 False {}
state, action, reward, done, info
[-0.02232731  0.40527418 -0.03632667 -0.5964274 ] 1 1.0 False {}
state, action, reward, done, info
[-0.01422183  0.60088517 -0.04825522 -0.90032799] 1 1.0 False {}
state, action, reward, done, info
[-0.00220412  0.79662662 -0.06626178 -1.20777993] 1 1.0 False {}
state, action, reward, done, info
[ 0.01372841  0.99253901 -0.09041738 -1.52047032] 1 1.0 False {}
state, action, reward, done, info
[ 0.03357919  1.18862999 -0.12082679 -1.83995178] 1 1.0 False {}


To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [5]:
print(rewards[-20:])
print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print(np.max(np.array(actions)), np.min(np.array(actions)))
print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print(np.max(np.array(rewards)), np.min(np.array(rewards)))
print(np.max(np.array(states)), np.min(np.array(states)))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
(10,) (10, 4) (10,) (10,)
float64 float64 int64 bool
1 0
2
1.0 1.0
1.1886299860716203 -1.8399517796564002


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [6]:
def model_input(state_size):
    # Input states/ observation or internal states
    states = tf.placeholder(tf.float32, [None, state_size], name=None)
    
    # Output actions
    actions = tf.placeholder(tf.int32, [None], name=None)

    # Target Q values for training
    targetQs = tf.placeholder(tf.float32, [None], name=None)
    return states, actions, targetQs

In [7]:
# tf.layers.batch_normalization(
#     inputs, ??????????????????????????????????????
#     axis=-1, okkkkkkkkkkkkkkkkkkkkkkkkkkkk
#     momentum=0.99, okkkkkkkkkkkkkkkkkkkkkkkk
#     epsilon=0.001, okkkkkkkkkkkkkkkkkkkkkkkkkkk
#     center=True, OOOOOOOOOOOOOOOOOOOOOOK
#     scale=True, OOOOOOOOOOOOOOOOOOOOOOOOOOOOOK
#     beta_initializer=tf.zeros_initializer(), OOOOOOOOOOOOOOOOOOOOOOOOOOOOK
#     gamma_initializer=tf.ones_initializer(), OOOOOOOOOOOOOOOOOOOOOOOOOOOOK
#     moving_mean_initializer=tf.zeros_initializer(), OOOOOOOOOOOOOOOOOOOOOOOOOK
#     moving_variance_initializer=tf.ones_initializer(), OOOOOOOOOOOOOOOOOOOOOOK
#     beta_regularizer=None,
#     gamma_regularizer=None,
#     beta_constraint=None,
#     gamma_constraint=None,
#     training=False, ??????????????????????????????
#     trainable=True, ???????????????????????????
#     name=None,
#     reuse=None,
#     renorm=False,
#     renorm_clipping=None,
#     renorm_momentum=0.99, OOOOOOOOOOOOOOOOOK
#     fused=None,
#     virtual_batch_size=None,
#     adjustment=None
# )

In [8]:
def generator(states, hidden_size, action_size, reuse=False, is_bn_training=True):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=is_bn_training)
        nl1 = tf.maximum(0.01 * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=is_bn_training)
        nl2 = tf.maximum(0.01 * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)
        # predictions = tf.tanh(logits)

        return logits

In [9]:
def model_loss(states, hidden_size, actions, action_size, targetQs):
    # Output layer
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)        

    # One hot encode the actions to later choose the Q-value for the action
    actions_labels = tf.one_hot(actions, action_size)

    # Train with loss (targetQ - Q)^2
    Qs = tf.reduce_sum(tf.multiply(actions_logits, actions_labels), axis=1)

    # error backpropagation: loss and opt
    loss = tf.reduce_mean(tf.square(targetQs - Qs))
    
    return actions_logits, Qs, loss

In [10]:
class DQN:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):
        
        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs, self.loss = model_loss(action_size=action_size, actions=self.actions, 
                                                      states=self.states, hidden_size=hidden_size, 
                                                      targetQs=self.targetQs)

        # Update the model: backward pass and backprop
        self.opt = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [11]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
            
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [13]:
train_episodes = 1000          # max number of episodes to learn from
max_steps = 200               # max steps in an episode
gamma = 0.99                   # future reward discount

# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
hidden_size = 64              # number of units in each Q-network hidden layer -- simulation
state_size = 4                # number of units for the input state/observation -- simulation
action_size = 2               # number of units for the output actions -- simulation

# Memory parameters
memory_size = 10000            # memory capacity
batch_size = 20                # experience mini-batch size
learning_rate = 0.001          # learning rate for training/optimization/adam

In [14]:
tf.reset_default_graph()
model = DQN(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

## Populate the experience memory

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [15]:
# Initialize the simulation
env.reset()

# Take one random step to get the pole and cart moving
state, reward, done, _ = env.step(env.action_space.sample())

# init memory
memory = Memory(max_size=memory_size)

# Make a bunch of random actions and store the experiences
for _ in range(batch_size):
    # Uncomment the line below to watch the simulation
    # env.render()

    # Make a random action
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)

    # Episode is done/failed or not;
    if done:
        # The simulation fails so no next state
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        memory.add((state, action, reward, next_state))
        
        # Start new episode
        env.reset()
        
        # Take one random step to get the pole and cart moving
        state, reward, done, _ = env.step(env.action_space.sample())
    else:
        # Add experience to memory
        memory.add((state, action, reward, next_state))
        state = next_state

## Training

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Now train with experiences
saver = tf.train.Saver()

# Total rewards and losses list for plotting
rewards_list, loss_list = [], []

# TF session for training
with tf.Session() as sess:
    
    # Initialize variables
    sess.run(tf.global_variables_initializer())

    # Training episodes/epochs
    step = 0
    for ep in range(train_episodes):
        
        # Env/agent steps/batches/minibatches
        total_reward = 0
        loss = 0
        t = 0
        while t < max_steps:
            step += 1
            
            # Uncomment this next line to watch the training
            # env.render() 
            
            # Explore or Exploit
            explore_p = explore_stop + (explore_start - explore_stop)*np.exp(-decay_rate*step) 
            if explore_p > np.random.rand():
                # Make a random action
                action = env.action_space.sample()
            else:
                # Get action from DQN (the inference mode)
                feed_dict = {model.states: state.reshape((1, *state.shape))}
                # actions_logits = sess.run(model.actions_logits, feed_dict)
                model_actions_logits = generator(action_size=action_size, hidden_size=hidden_size, 
                                                 is_bn_training=False, reuse=True, states=model.states)
                actions_logits = sess.run(model_actions_logits, feed_dict)
                action = np.argmax(actions_logits)
            
            # Take action, get new state and reward
            next_state, reward, done, _ = env.step(action)
    
            # Cumulative reward
            total_reward += reward
            
            # Episode/epoch training is done/failed!
            if done:
                # the episode ends so no next state
                next_state = np.zeros(state.shape)
                t = max_steps
                
                print('Episode: {}'.format(ep),
                      'Total reward: {}'.format(total_reward),
                      'Training loss: {:.4f}'.format(loss),
                      'Explore P: {:.4f}'.format(explore_p))
                
                # total rewards and losses for plotting
                rewards_list.append((ep, total_reward))
                loss_list.append((ep, loss))
                
                # Add experience to memory
                memory.add((state, action, reward, next_state))
                
                # Start new episode
                env.reset()
                
                # Take one random step to get the pole and cart moving
                state, reward, done, _ = env.step(env.action_space.sample())

            else:
                # Add experience to memory
                memory.add((state, action, reward, next_state))
                state = next_state
                t += 1
            
            # Sample mini-batch from memory
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            rewards = np.array([each[2] for each in batch])
            next_states = np.array([each[3] for each in batch])
            
            # Model in the inference mode
            feed_dict={model.states: next_states}
            # next_actions_logits = sess.run(model.actions_logits, feed_dict)
            model_next_actions_logits = generator(action_size=action_size, hidden_size=hidden_size, 
                                             is_bn_training=False, reuse=True, states=model.states)
            next_actions_logits = sess.run(model_next_actions_logits, feed_dict)

            # Set target_Qs to 0 for states where episode ends
            episode_ends = (next_states == np.zeros(states[0].shape)).all(axis=1)
            next_actions_logits[episode_ends] = (0, 0)

            # Bellman equation
            targetQs = rewards + (gamma * np.max(next_actions_logits, axis=1))

            # Updating the model (the training mode)
            loss, _ = sess.run([model.loss, model.opt],
                                feed_dict={model.states: states,
                                           model.targetQs: targetQs,
                                           model.actions: actions})
            
            #             # Printing each step before episode is done.
            #             print('Env/Rob Step: {}'.format(step),
            #                   'Total reward: {}'.format(total_reward),
            #                   'Each reward: {}'.format(reward),
            #                   'Average reward: {}'.format(np.mean(rewards)),
            #                   'Training loss: {:.4f}'.format(loss),
            #                   'Explore P: {:.4f}'.format(explore_p))

    # Save the trained model 
    saver.save(sess, "checkpoints/DQN-cartpole.ckpt")

Episode: 0 Total reward: 16.0 Training loss: 0.7616 Explore P: 0.9984
Episode: 1 Total reward: 16.0 Training loss: 0.3490 Explore P: 0.9968
Episode: 2 Total reward: 13.0 Training loss: 0.0975 Explore P: 0.9956
Episode: 3 Total reward: 16.0 Training loss: 0.1076 Explore P: 0.9940
Episode: 4 Total reward: 15.0 Training loss: 0.0626 Explore P: 0.9925
Episode: 5 Total reward: 16.0 Training loss: 0.1440 Explore P: 0.9909
Episode: 6 Total reward: 12.0 Training loss: 0.0511 Explore P: 0.9898
Episode: 7 Total reward: 19.0 Training loss: 0.0213 Explore P: 0.9879
Episode: 8 Total reward: 18.0 Training loss: 0.0342 Explore P: 0.9861
Episode: 9 Total reward: 16.0 Training loss: 0.0331 Explore P: 0.9846
Episode: 10 Total reward: 16.0 Training loss: 0.0277 Explore P: 0.9830
Episode: 11 Total reward: 11.0 Training loss: 0.0149 Explore P: 0.9820
Episode: 12 Total reward: 18.0 Training loss: 0.0507 Explore P: 0.9802
Episode: 13 Total reward: 15.0 Training loss: 0.0105 Explore P: 0.9787
Episode: 14 Tota

Episode: 116 Total reward: 67.0 Training loss: 0.0853 Explore P: 0.7339
Episode: 117 Total reward: 15.0 Training loss: 0.0089 Explore P: 0.7328
Episode: 118 Total reward: 24.0 Training loss: 0.0219 Explore P: 0.7310
Episode: 119 Total reward: 44.0 Training loss: 0.0294 Explore P: 0.7279
Episode: 120 Total reward: 19.0 Training loss: 0.0549 Explore P: 0.7265
Episode: 121 Total reward: 24.0 Training loss: 0.0106 Explore P: 0.7248
Episode: 122 Total reward: 67.0 Training loss: 0.0377 Explore P: 0.7200
Episode: 123 Total reward: 31.0 Training loss: 0.0505 Explore P: 0.7178
Episode: 124 Total reward: 19.0 Training loss: 0.0180 Explore P: 0.7165
Episode: 125 Total reward: 16.0 Training loss: 0.0516 Explore P: 0.7154
Episode: 126 Total reward: 9.0 Training loss: 0.0246 Explore P: 0.7147
Episode: 127 Total reward: 140.0 Training loss: 0.0863 Explore P: 0.7049
Episode: 128 Total reward: 27.0 Training loss: 0.0216 Explore P: 0.7031
Episode: 129 Total reward: 52.0 Training loss: 0.0064 Explore P:

Episode: 230 Total reward: 26.0 Training loss: 0.2225 Explore P: 0.5802
Episode: 231 Total reward: 9.0 Training loss: 0.0747 Explore P: 0.5796
Episode: 232 Total reward: 15.0 Training loss: 0.1911 Explore P: 0.5788
Episode: 233 Total reward: 11.0 Training loss: 0.0785 Explore P: 0.5782
Episode: 234 Total reward: 14.0 Training loss: 0.2916 Explore P: 0.5774
Episode: 235 Total reward: 22.0 Training loss: 0.1525 Explore P: 0.5761
Episode: 236 Total reward: 10.0 Training loss: 0.1511 Explore P: 0.5756
Episode: 237 Total reward: 14.0 Training loss: 0.0342 Explore P: 0.5748
Episode: 238 Total reward: 8.0 Training loss: 0.0898 Explore P: 0.5743
Episode: 239 Total reward: 7.0 Training loss: 0.1626 Explore P: 0.5739
Episode: 240 Total reward: 15.0 Training loss: 0.2610 Explore P: 0.5731
Episode: 241 Total reward: 13.0 Training loss: 0.0267 Explore P: 0.5723
Episode: 242 Total reward: 14.0 Training loss: 0.1083 Explore P: 0.5716
Episode: 243 Total reward: 10.0 Training loss: 0.1415 Explore P: 0.

Episode: 345 Total reward: 8.0 Training loss: 0.2140 Explore P: 0.5036
Episode: 346 Total reward: 13.0 Training loss: 0.1221 Explore P: 0.5030
Episode: 347 Total reward: 14.0 Training loss: 0.3038 Explore P: 0.5023
Episode: 348 Total reward: 10.0 Training loss: 0.0467 Explore P: 0.5018
Episode: 349 Total reward: 8.0 Training loss: 0.0612 Explore P: 0.5014
Episode: 350 Total reward: 14.0 Training loss: 0.3576 Explore P: 0.5007
Episode: 351 Total reward: 9.0 Training loss: 0.4422 Explore P: 0.5003
Episode: 352 Total reward: 15.0 Training loss: 0.1296 Explore P: 0.4996
Episode: 353 Total reward: 10.0 Training loss: 0.1572 Explore P: 0.4991
Episode: 354 Total reward: 10.0 Training loss: 0.2122 Explore P: 0.4986
Episode: 355 Total reward: 13.0 Training loss: 0.1977 Explore P: 0.4979
Episode: 356 Total reward: 10.0 Training loss: 0.2643 Explore P: 0.4975
Episode: 357 Total reward: 7.0 Training loss: 0.2616 Explore P: 0.4971
Episode: 358 Total reward: 8.0 Training loss: 0.0880 Explore P: 0.49

Episode: 460 Total reward: 11.0 Training loss: 0.2542 Explore P: 0.4435
Episode: 461 Total reward: 10.0 Training loss: 0.2501 Explore P: 0.4431
Episode: 462 Total reward: 18.0 Training loss: 0.1485 Explore P: 0.4423
Episode: 463 Total reward: 11.0 Training loss: 0.2703 Explore P: 0.4418
Episode: 464 Total reward: 12.0 Training loss: 0.3525 Explore P: 0.4413
Episode: 465 Total reward: 11.0 Training loss: 0.1527 Explore P: 0.4408
Episode: 466 Total reward: 9.0 Training loss: 0.2077 Explore P: 0.4404
Episode: 467 Total reward: 8.0 Training loss: 0.1860 Explore P: 0.4401
Episode: 468 Total reward: 9.0 Training loss: 0.2535 Explore P: 0.4397
Episode: 469 Total reward: 8.0 Training loss: 0.2243 Explore P: 0.4394
Episode: 470 Total reward: 8.0 Training loss: 0.1909 Explore P: 0.4390
Episode: 471 Total reward: 14.0 Training loss: 0.9314 Explore P: 0.4384
Episode: 472 Total reward: 10.0 Training loss: 0.1803 Explore P: 0.4380
Episode: 473 Total reward: 12.0 Training loss: 0.2547 Explore P: 0.43

Episode: 575 Total reward: 11.0 Training loss: 0.2216 Explore P: 0.3947
Episode: 576 Total reward: 12.0 Training loss: 0.0699 Explore P: 0.3942
Episode: 577 Total reward: 9.0 Training loss: 0.1281 Explore P: 0.3939
Episode: 578 Total reward: 13.0 Training loss: 0.2448 Explore P: 0.3934
Episode: 579 Total reward: 7.0 Training loss: 0.4646 Explore P: 0.3931
Episode: 580 Total reward: 12.0 Training loss: 0.0481 Explore P: 0.3926
Episode: 581 Total reward: 11.0 Training loss: 0.2980 Explore P: 0.3922
Episode: 582 Total reward: 16.0 Training loss: 0.0724 Explore P: 0.3916
Episode: 583 Total reward: 8.0 Training loss: 0.2124 Explore P: 0.3913
Episode: 584 Total reward: 15.0 Training loss: 0.1790 Explore P: 0.3907
Episode: 585 Total reward: 17.0 Training loss: 0.2244 Explore P: 0.3901
Episode: 586 Total reward: 10.0 Training loss: 0.2370 Explore P: 0.3897
Episode: 587 Total reward: 9.0 Training loss: 0.0909 Explore P: 0.3894
Episode: 588 Total reward: 12.0 Training loss: 0.3927 Explore P: 0.3

Episode: 690 Total reward: 7.0 Training loss: 0.4260 Explore P: 0.3526
Episode: 691 Total reward: 9.0 Training loss: 0.6688 Explore P: 0.3523
Episode: 692 Total reward: 14.0 Training loss: 0.1655 Explore P: 0.3519
Episode: 693 Total reward: 10.0 Training loss: 0.3231 Explore P: 0.3515
Episode: 694 Total reward: 8.0 Training loss: 0.4961 Explore P: 0.3512
Episode: 695 Total reward: 13.0 Training loss: 0.1151 Explore P: 0.3508
Episode: 696 Total reward: 8.0 Training loss: 0.1617 Explore P: 0.3505
Episode: 697 Total reward: 10.0 Training loss: 0.1438 Explore P: 0.3502
Episode: 698 Total reward: 11.0 Training loss: 0.9046 Explore P: 0.3498
Episode: 699 Total reward: 11.0 Training loss: 0.2217 Explore P: 0.3494
Episode: 700 Total reward: 9.0 Training loss: 0.5895 Explore P: 0.3491
Episode: 701 Total reward: 7.0 Training loss: 0.5216 Explore P: 0.3489
Episode: 702 Total reward: 15.0 Training loss: 0.6295 Explore P: 0.3484
Episode: 703 Total reward: 10.0 Training loss: 0.3178 Explore P: 0.348

## Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, rews = np.array(rewards_list).T
smoothed_rews = running_mean(rews, 10)
plt.plot(eps[-len(smoothed_rews):], smoothed_rews)
plt.plot(eps, rews, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total Reward')

In [ ]:
eps, loss = np.array(loss_list).T
smoothed_loss = running_mean(loss, 10)
plt.plot(eps[-len(smoothed_loss):], smoothed_loss)
plt.plot(eps, loss, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Training Loss')

## Testing

Let's checkout how our trained agent plays the game.

In [19]:
test_episodes = 10
test_max_steps = 10000000
env.reset()
with tf.Session() as sess:
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    # Restore/load the trained model 
    saver.restore(sess, "checkpoints/DQN-cartpole.ckpt")
    
    # Training episodes/ epochs
    for ep in range(test_episodes):
        
        # Env/Rob steps for minibatches/batches training
        t = 0
        while t < test_max_steps:
            
            # render the graphics
            env.render() 
            
            # Get action from DQN
            feed_dict = {model.states: state.reshape((1, *state.shape))}
            actions_logits = sess.run(model.actions_logits, feed_dict)
            action = np.argmax(actions_logits)
            
            # Take action, get new state and reward
            next_state, reward, done, _ = env.step(action)
            
            # If episode is done/failed;
            if done:
                t = test_max_steps
                env.reset()
                # Take one random step to get the pole and cart moving
                state, reward, done, _ = env.step(env.action_space.sample())

            else:
                state = next_state
                t += 1

# Closing the simulated env
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/DQN-cartpole.ckpt


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.